<center>

# **DATA PROCESSING**

</center>

With this spcrip we:

**Step 1:**
Concatenate all the files created when we gather the data.
Create a new column in each folder called "clean_text" and save in a new folder (Cleaned_data) the files that do not have any error.

The cleaning process:

-	Remove twits with NaN in the text.
-	Remove # and separtate hastags (CaliforniaWildfire = California Wildfire)
-	Remove URL
-	Remove special characters and punctuation.
- Lowercassing
-	Remove stop words, however we exclude negation stop words because those words provide us information about negative sentiments.
-	Tokenization

**Step 2:**
We join all the files in only one.

**Step 3:**
A little more cleaning
-	Remove duplicated twits.
-	Remove tweets with a date in the text.
- Filter twitts in english only

At the end, this script create a concatenated file with all the cleaned results. The name of the file is "WildfireName_data.csv" and it is saved in the folder DataProcessing

In [ ]:
import os
import glob
import pandas as pd
import re
import csv
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [ ]:
#Mount your Google Drive to Colab

from google.colab import drive
drive.mount('/content/drive')

Chose the wildifire to analyze

In [ ]:
#name for the final file
Wildfire_name = '1.TubbsFire'
folder        = 'ResultsTubbsFire/'
final_folder  = 'ResultsTubbsFire/Cleaned_files/'


Set folder and output paths

In [ ]:
folder_path    = '/content/drive/MyDrive/Mental_Health_Wildfire/Twitter_Data/Tubbs_Codes/1.CollectingData/'+folder  # change this to the path of the folder containing the CSV files
output_path    = '/content/drive/MyDrive/Mental_Health_Wildfire/Twitter_Data/Tubbs_Codes/1.CollectingData/'+final_folder  # change this to the desired output file path

os.listdir('/content/drive/MyDrive/Mental_Health_Wildfire/Twitter_Data/Tubbs_Codes/1.CollectingData/')

In [ ]:
# Define the data columns types
dtypes = {
    "tweet_id": "object",
    "tweet_text":"object",
    "tweet_possibly_sensitive": "object",
    "tweet_text": "object",
    "tweet_source": "object",
    "tweet_lang": "object",
    "tweet_retweet_count": "object",
    "tweet_reply_count":"object",
    "tweet_like_count": "object",
    "tweet_quote_count": "object",
    "tweet_impression_count": "object",
    "user_id":"object",
    "user_username": "object",
    "user_verified":"object",
    "user_protected":"object",
    "user_description":"object",
    "user_profile_image_url":"object",
    "user_location":"object",
    "user_followers_count":"object",
    "user_friends_count":"object",
    "user_tweet_count":"object",
    "place_id":"object",
    "place_name": "object",
    "place_full_name":"object",
    "place_country":"object",
    "place_country_code":"object",
    "place_type":"object"
}

## **Step 1:** First clean phase

-	Remove twits with NaN in the text.
- Remove # and separtate hastags (CaliforniaWildfire = California Wildfire)
-	Remove URL
-	Remove special characters and punctuation.
-	Remove stop words, however we exclude negation stop words because those words provide us information about negative sentiments.
-	Tokenization



In [ ]:
def preprocess_text(text):

    #separate words in the hashtagas and remove #
    hashtags = re.findall(r'#\w+', text)

    # Iterate over hashtags and separate words
    for hashtag in hashtags:
        separated_words = re.sub(r'([a-z])([A-Z])', r'\1 \2', hashtag[1:])

        # Replace the hashtag with separated words
        text = text.replace(hashtag, separated_words)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    # Remove special characters and punctuation
    text = re.sub('[^a-zA-Z0-9\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)
    # Lowercasing
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('english'))

    # Define stopwords related to negation
    negation_stopwords = {'not', 'no', 'don', "don't", 'won', "won't", 'cannot', 'couldn', "couldn't",'aren',
     "aren't",'didn', "didn't", 'doesn', "doesn't",'hadn', "hadn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'needn',  "needn't",  'nor', 'weren',  "weren't", 'weren',  "weren't",'wouldn',
    "wouldn't", 'hasn', "hasn't",'mustn', "mustn't",'shan', "shan't", 'shouldn',  "shouldn't", 'wasn',  "wasn't"}

    # Exclude negation stopwords from the set
    filtered_stop_words = stop_words - negation_stopwords
    tokens = [token for token in tokens if not token in stop_words]
    return ' '.join(tokens)

In [ ]:
#The files are saved in a new folder called Cleaned_files

list_bad_file_name = []

for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        try:
            file_path = os.path.join(folder_path, file_name)
            file_size = os.stat(file_path).st_size
            if file_size == 0:
                print("Error: Empty file -", file_name, "- Skipping the file.")
                list_bad_file_name.append(file_name)
                continue

            with open(file_path) as f:
                first_line = f.readline().strip()  # Read the first line
            if not first_line:  # Empty first line
                print("Error: No columns to parse from file -", file_name, "- Skipping the file.")
                list_bad_file_name.append(file_name)
                continue

            df = pd.read_csv(file_path)

            #remove twitts with NaN in the text
            df= df.dropna(subset=['tweet_text'])

            #Remove RT
            df = df[~df['tweet_text'].str.startswith('RT')]


            # convert columns to desired data types
            df["tweet_id"]                   = df["tweet_id"].astype(object)
           # df["tweet_created_at"]           = pd.to_datetime(df["tweet_created_at"])
            df["tweet_created_at"]           = df["tweet_created_at"].astype(object)
            df["tweet_possibly_sensitive"]   = df["tweet_possibly_sensitive"].astype(object)
            df["tweet_text"]                 = df["tweet_text"].astype(object)
            df["tweet_source"]               = df["tweet_source"].astype(object)
            df["tweet_lang"]                 = df["tweet_lang"].astype(object)
            df["tweet_retweet_count"]        = df["tweet_retweet_count"].astype(object)
            df["tweet_reply_count"]          = df["tweet_reply_count"].astype(object)
            df["tweet_like_count"]           = df["tweet_like_count"].astype(object)
            df["tweet_quote_count"]          = df["tweet_quote_count"].astype(object)
            df["tweet_impression_count"]     = df["tweet_impression_count"].astype(object)
            df["user_id"]                    = df["user_id"].astype(object)
            df["user_username"]              = df["user_username"].astype(object)
            df["user_verified"]              = df["user_verified"].astype(object)
            df["user_protected"]             = df["user_protected"].astype(object)
            df["user_description"]           = df["user_description"].astype(object)
            df["user_profile_image_url"]     = df["user_profile_image_url"].astype(object)
            df["user_location"]              = df["user_location"].astype(object)
            df["user_followers_count"]       = df["user_followers_count"].astype(object)
            df["user_friends_count"]         = df["user_friends_count"].astype(object)
            df["user_tweet_count"]           = df["user_tweet_count"].astype(object)
            df["place_id"]                   = df["place_id"].astype(object)
            df["place_name"]                 = df["place_name"].astype(object)
            df["place_full_name"]            = df["place_full_name"].astype(object)
            df["place_country"]              = df["place_country"].astype(object)
            df["place_country_code"]         = df["place_country_code"].astype(object)
            df["place_type"]                 = df["place_type"].astype(object)

            #df["tweet_text"] = df["tweet_text"].apply(lambda x: re.sub(pattern, "", x))

            df['clean_text']  = df['tweet_text'].apply(preprocess_text)
            output_file_name  = "cleaned_" + file_name
            df.to_csv(os.path.join(output_path, output_file_name), index=False)
        except pd.errors.ParserError:
            list_bad_file_name.append(file_name)
            print("Error in file:", file_name, "- Skipping the file.")

## **Step 2:**  Create only one file.

We take all the files saved in cleaned_files to create only one.
The new file is saved in the *DataProcessing* folder

In [ ]:
# Create an empty list to store DataFrames
#output_path    = '/content/drive/MyDrive/Mental_Health_Wildfire/Twitter_Data/Tubbs_Codes/1.Collecting_data/'+final_folder  # change this to the desired output file path
dfs = []

# Loop through each CSV file in the folder
for file_name in os.listdir(output_path):
    if file_name.endswith(".csv"):
        # Load the CSV file into a Pandas DataFrame
        df = pd.read_csv(os.path.join(output_path, file_name),dtype=dtypes)
        #print(df.shape)

        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
df_concatenated = pd.concat(dfs, ignore_index=True)

print(df_concatenated.shape)
df_original = df_concatenated

df_original.shape

# **Step 3:** A litle more clining

In [ ]:
#remove twitts with NaN in the text
df_concatenated= df_concatenated.dropna(subset=['clean_text'])

#remove dupplicated twitts
df_concatenated = df_concatenated.drop_duplicates(subset=['tweet_id'], keep='first')

#remove tweets with a date in the text
# Define a regular expression pattern to match the format of a date
date_pattern = re.compile(r"\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\+\d{2}:\d{2}")

# Iterate through each tweet in the DataFrame
for index, row in df_concatenated.iterrows():
    tweet_text = row["tweet_text"]

    # Check if the tweet text matches the date pattern
    if date_pattern.match(tweet_text):
        # Remove the tweet from the DataFrame
        df_concatenated.drop(index, inplace=True)

print(df_concatenated.shape)

In [ ]:
# Filter out tweets starting with 'RT'
df_concatenated = df_concatenated[~df_concatenated['tweet_text'].str.startswith('RT')]
df_concatenated.head(5)
print(df_concatenated.shape)

# * Check languages

We are going to keep only tweets in english

In [ ]:
filtered_df2 = df_concatenated
#check the languages
column_data = filtered_df2['tweet_lang']

# Filter out missing values and convert non-string values to strings
column_data = column_data.dropna().astype(str)

# Join the column values and split into individual words
all_words = ' '.join(column_data).split()

# Create a set of unique words
unique_words = set(all_words)

# Display the unique words
print(unique_words)

In [ ]:
#use this line to see the texts of specific language observed in the previous line of code

filtered_df2[(filtered_df2['tweet_lang'].str.contains('en', na=False))]

In [ ]:
# Find the index position by index label and remove data in 'it'
index_position = filtered_df2.loc[filtered_df2['tweet_lang'].str.contains('en',na=False)].index
df_english     = filtered_df2.loc[index_position]

print(len(index_position))
print(df_english.shape)

# * Tokenization

Save the tokens in a new column called "tokens"

In [ ]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk

In [ ]:
# Tokenize and preprocess the text
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

#Remove na from clean text
df_english           = df_english.dropna(subset=['clean_text'])
df_english['tokens'] = df_english['clean_text'].apply(tokenize)

In [ ]:
df_english['tokens']
df_english.shape

In [ ]:
# Save the file in Data folder
join_files_path= '/content/drive/MyDrive/Mental_Health_Wildfire/Twitter_Data/Tubbs_Codes/Data/'

output_file_name = Wildfire_name+"_version1.csv"
df_english.to_csv(os.path.join(join_files_path,output_file_name), index=False)

In [ ]:
join_files_path

In [ ]:
df_english['tweet_created_at'].max()

df_english.shape

**Save file:**


In [ ]:
# Save the concatenated DataFrame to a new CSV file

output_file_name2 = "1.Tubbs_version1.csv"
df_english.to_csv(os.path.join(join_files_path,output_file_name2), index=False)